In [1]:
import numpy as np
import math
import MaxLik as ML

# Basic ideas of WP tomography:

When an unknown polarization state impinges the WP analyzer followed by a polarizer and a photodetector, the analyzer performs a polarization projection to the particular state (H, V, D, A, R, and L, required WP angles are known). Six detection responses are sequentially acquired for the corresponding projections. The input polarization state could be easily retrieved by solving a set of resulting linear equations or, which is how we proceed, using the maximum-likelihood method.

In [2]:
sigma1 = np.array([[1., 0.], [0., -1.]])
sigma2 = np.array([[0., 1.], [1., 0.]])
sigma3 = np.array([[0., -1j], [1j, 0.]])
sigma = np.array([sigma1, sigma2, sigma3])

blochFromRho = lambda rho_ : np.real(np.array([
    np.trace(np.dot(rho_,sigma1)),
    np.trace(np.dot(rho_,sigma2)),
    np.trace(np.dot(rho_,sigma3))
]))

rhoFromBloch = lambda bloch_ : np.array([
    [1+bloch_[0], bloch_[1]-1j*bloch_[2]],
    [bloch_[1]+1j*bloch_[2], 1-bloch_[0]]
])/2.

In [3]:
#Idela HVDARL
H = np.array([[1],[0]])
V = np.array([[0],[1]])
D = 1/math.sqrt(2) * (H + V)
A = 1/math.sqrt(2) * (H - V)
R = 1/math.sqrt(2) * (H + 1j*V)
L = 1/math.sqrt(2) * (H - 1j*V)

idealStates = [H,V,D,A,R,L]

In [4]:
def ReturnRho(jones):
    return (jones@np.transpose(np.conjugate(jones)))

In [5]:
blochFromRho(ReturnRho(L))

array([ 0.,  0., -1.])

# Polarisation elements

In [6]:
linPol = np.array([[1,0],[0,0]])
QWP = np.array([[1,0],[0,-1j]])*np.exp(1j*np.pi/4)
HWP = 1/math.sqrt(2)*np.array([[1,0],[0,-1]])

#Angles for WP analysis
angles = {'H': [   0   ,   0],
          'V': [  45   ,   0],
          'D': [  22.5 ,   0],
          'A': [ -22.5 ,   0],
          'R': [  22.5 , -45],
          'L': [  22.5 ,  45],
         }

# Analysis

In [7]:
#analyzed state |IN>
inState = L

#number of photons used for analysis
photons = 100000

In [8]:
def Rot(phi):
    return(np.array([[np.cos(phi),-np.sin(phi)],[np.sin(phi),np.cos(phi)]]))

QWP = np.array([[1,0],[0,-1j]])
HWP = np.array([[1,0],[0,-1]])

In [9]:
def OutState(argument):
    [phiHWP,phiQWP] = argument
    phiHWP = math.radians(phiHWP)
    phiQWP = math.radians(phiQWP)
    return((linPol@(Rot(-phiQWP)@QWP@Rot(phiQWP))@(Rot(-phiHWP)@HWP@Rot(phiHWP)))@inState)

In [10]:
#proj = 'R'
#outState = OutState(angles[proj])
#print (outState)
#print (abs(np.transpose(np.conjugate(outState))@outState)[0][0])

In [11]:
projList = ['H', 'V', 'D', 'A', 'R', 'L']
data = []

for proj in projList:
    outState = OutState(angles[proj])
    data.append(abs(np.transpose(outState)@outState)[0][0])
    
#print (data)

In [12]:
prob = data/sum(data)
countData = np.random.multinomial(photons,prob) #photons per projection - all projections measured at the same time (model)

print ('Photons per proj.: ' + str(countData))

Photons per proj.: [16786 16753 16573 16437     0 33451]


In [13]:
def Proj(argument):
    [phiHWP,phiQWP] = argument
    phiHWP = math.radians(phiHWP)
    phiQWP = math.radians(phiQWP)
    return((Rot(phiHWP)@HWP@Rot(-phiHWP))@(Rot(phiQWP)@QWP@Rot(-phiQWP))@H)

projectors = []
for proj in projList:
    projectors.append(np.outer(Proj(angles[proj]),Proj(angles[proj]).conj()))

## Maximum-likelihood

In [14]:
[rho,bloch] = ML.Maxlik(countData, projectors)

# Result

In [15]:
print ('Analyzed state: ' + str(bloch))
print ('|IN>:           ' + str(blochFromRho(ReturnRho(inState))))

Analyzed state: [ 6.56525771e-04  2.73446674e-03 -9.99996046e-01]
|IN>:           [ 0.  0. -1.]


In [16]:
#Proj([45,45])